In [ ]:
import pytest
from unittest.mock import patch, MagicMock
from rss_collector.feed_parser import retry_with_backoff

def test_retry_with_backoff_success():
    """
    Test that retry_with_backoff succeeds after a few failed attempts.
    """
    test_url = "https://example.com/rss"

    # Mock `fetch_feed_content_with_proxy` to fail twice before succeeding
    with patch("rss_collector.feed_parser.fetch_feed_content_with_proxy") as mock_fetch:
        # First two attempts fail, third succeeds
        mock_fetch.side_effect = [None, None, "<rss>Valid Feed</rss>"]

        # Mock feedparser.parse to return a valid feed
        with patch("rss_collector.feed_parser.feedparser.parse") as mock_parse:
            mock_parse.return_value = MagicMock(feed={"title": "Test Feed"}, entries=[])

            parsed_feed = retry_with_backoff(test_url, retries=3)

            # Assert that the fetch function was called 3 times
            assert mock_fetch.call_count == 3
            assert parsed_feed is not None
            assert parsed_feed.feed["title"] == "Test Feed"

def test_retry_with_backoff_failure():
    """
    Test that retry_with_backoff returns None after all retries fail.
    """
    test_url = "https://example.com/rss"

    # Mock `fetch_feed_content_with_proxy` to always fail
    with patch("rss_collector.feed_parser.fetch_feed_content_with_proxy") as mock_fetch:
        mock_fetch.return_value = None

        parsed_feed = retry_with_backoff(test_url, retries=3)

        # Assert that the fetch function was called 3 times
        assert mock_fetch.call_count == 3
        assert parsed_feed is None


In [ ]:
# File: tests/test_feed_parser.py

import pytest
from rss_collector.feed_parser import retry_with_backoff
from rss_collector.proxy import fetch_feed_content_with_proxy

def test_retry_with_backoff(mocker):
    """
    Test the retry_with_backoff function with mocked fetch_feed_content_with_proxy.
    """
    # Mock the fetch_feed_content_with_proxy function to simulate transient failures
    mock_fetch = mocker.patch(
        "rss_collector.proxy.fetch_feed_content_with_proxy",
        side_effect=[None, None, "<rss>Mocked RSS Content</rss>"]
    )

    # Mock the feedparser.parse function to simulate a valid parsed feed
    mock_feedparser = mocker.patch(
        "rss_collector.feed_parser.feedparser.parse",
        return_value={"feed": {"title": "Mock Feed"}, "entries": [{"title": "Mock Entry"}]}
    )

    # Test the function
    test_url = "https://example.com/rss"
    parsed_feed = retry_with_backoff(test_url, retries=3, delay=1)

    # Assertions
    assert parsed_feed is not None
    assert parsed_feed["feed"]["title"] == "Mock Feed"
    assert len(parsed_feed["entries"]) == 1
    assert parsed_feed["entries"][0]["title"] == "Mock Entry"

    # Ensure the fetch function was called three times (2 failures + 1 success)
    assert mock_fetch.call_count == 3
